In [2]:
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2

In [46]:
#define base_directory where Alzheimer_s Dataset is located
base_directory = 'Alzheimer_s Dataset'
#base_directory = '/content/drive/MyDrive/' + base_directory #for google drive

In [47]:
# For reproducability
seed = 12222022
tf.random.set_seed(seed)
np.random.seed(seed)

image_generator = ImageDataGenerator(rescale=1/255, validation_split=0)
train_ds = image_generator.flow_from_directory(batch_size=16,
                                                 directory= base_directory+'/train',
                                                 shuffle=True,
                                                 target_size=(224,224), 
                                                 subset="training",
                                                 class_mode='categorical')
image_generator = ImageDataGenerator(rescale=1/255,validation_split=0.2) 
val_ds = image_generator.flow_from_directory(batch_size=16,
                                                 directory= base_directory+'/test',
                                                 shuffle=True,
                                                 target_size=(224,224),
                                                 class_mode='categorical')

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [6]:
base_model = MobileNetV2(input_shape=(224,224,3), weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False
x = Flatten()(base_model.output)

prediction = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=prediction)

model.compile(optimizer='adam',
loss=tf.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc'),'acc'])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=8,
                                            restore_best_weights=True)

model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=callback)

9420800/9406464 [==============================] - 1s 0us/step
Epoch 1/50
321/321 [==============================] - 81s 246ms/step - loss: 2.9816 - auc: 0.8264 - acc: 0.6395 - val_loss: 3.8816 - val_auc: 0.8190 - val_acc: 0.6388
Epoch 2/50
321/321 [==============================] - 76s 238ms/step - loss: 0.7054 - auc: 0.9570 - acc: 0.8508 - val_loss: 3.8375 - val_auc: 0.8124 - val_acc: 0.6427
Epoch 3/50
321/321 [==============================] - 76s 237ms/step - loss: 0.6618 - auc: 0.9614 - acc: 0.8746 - val_loss: 2.7144 - val_auc: 0.8336 - val_acc: 0.6638
Epoch 4/50
321/321 [==============================] - 76s 236ms/step - loss: 0.3280 - auc: 0.9836 - acc: 0.9250 - val_loss: 3.9581 - val_auc: 0.8199 - val_acc: 0.6638
Epoch 5/50
321/321 [==============================] - 76s 236ms/step - loss: 0.3656 - auc: 0.9822 - acc: 0.9293 - val_loss: 3.3383 - val_auc: 0.8415 - val_acc: 0.6873
Epoch 6/50
321/321 [==============================] - 74s 231ms/step - loss: 0.8249 - auc: 0.9636 - ac

In [7]:
export_path = "models/output"
model.save(export_path+'AlzheimersMRI.h5')

C:\Users\micha\AppData\Roaming\Python\Python38\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [21]:
from tensorflow.keras.preprocessing.image import load_img , img_to_array

from PIL import Image

In [48]:
#picking from train to confirm classes notation
filename = base_directory+"/train/ModerateDemented/moderateDem18.jpg"
img = load_img(filename , color_mode="rgb",target_size = (224 , 224))
img = img_to_array(img)
img = img.reshape(1 , 224 ,224 ,3)
img = img.astype('float32')
img = img/255.0
result = model.predict(img)

In [49]:
print(result)

[[3.44848772e-08 1.00000000e+00 1.04829534e-14 7.61163712e-15]]


In [50]:
classes = ["mild","moderate","very_mild","non"]
dict_result = {}
for i in range(4):
    dict_result[result[0][i]] = classes[i]
res = result[0]
res.sort()
res = res[::-1]
prob = res[:3]

In [51]:
prob_result = []
class_result = []
for i in range(3):
    prob_result.append((prob[i]*100).round(2))
    class_result.append(dict_result[prob[i]])

In [52]:
print(prob_result)
print(class_result)

[100.0, 0.0, 0.0]
['moderate', 'mild', 'very_mild']


In [53]:
print(dict_result[prob[0]], " plan with a ",(prob[0]*100).round(2), " probability")

moderate  plan with a  100.0  probability


In [60]:
#picking from train to confirm classes notation
filename = base_directory+"/test/NonDemented/26 (69).jpg"
img = load_img(filename , color_mode="rgb",target_size = (224 , 224))
img = img_to_array(img)
img = img.reshape(1 , 224 ,224 ,3)
img = img.astype('float32')
img = img/255.0
result = model.predict(img)

classes = ["mild","moderate","very_mild","non"]
dict_result = {}
for i in range(4):
    dict_result[result[0][i]] = classes[i]
res = result[0]
res.sort()
res = res[::-1]
prob = res[:3]
print(dict_result[prob[0]], " plan with a ",(prob[0]*100).round(2), " probability")

very_mild  plan with a  97.84  probability
